<a href="https://www.kaggle.com/code/amirmotefaker/pytorch-vs-numpy?scriptVersionId=122588093" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# NOTE

At its core, [PyTorch](https://pytorch.org/) provides two main features:

- An n-dimensional Tensor, similar to [NumPy](https://numpy.org/) but can run on GPUs

- Automatic differentiation for building and training neural networks

We will use a problem of fitting y=sin(x) with a third-order polynomial as our running example. The network will have four parameters and will be trained with gradient descent to fit random data by minimizing the [Euclidean distance](https://www.cuemath.com/euclidean-distance-formula/) between the network output and the true output.

# NumPy

- NumPy provides an n-dimensional array object and many functions for manipulating these arrays. Numpy is a generic framework for scientific computing; it does not know anything about computation graphs, or deep learning, or gradients. However, we can easily use numpy to fit a third-order polynomial to a sine function by manually implementing the forward and backward passes through the network using numpy operations.

In [1]:
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 753.2240098168371
199 504.8102195908351
299 339.4491546866778
399 229.32750006436947
499 155.95995938291207
599 107.05675101458183
699 74.44428358375795
799 52.68452922293922
899 38.15806934911251
999 28.454917632110416
1099 21.9696912210644
1199 17.63249501993083
1299 14.729962145348502
1399 12.786204490662854
1499 11.483587129944231
1599 10.609983626726013
1699 10.023646141650719
1799 9.629797134128056
1899 9.365024198369582
1999 9.186871682801566
Result: y = -0.011270413260048285 + 0.8411773220609123 x + 0.0019443338278444797 x^2 + -0.09111661370485116 x^3


# PyTorch

- Numpy is a great framework, but it cannot utilize GPUs to accelerate its numerical computations. For modern deep neural networks, GPUs often provide speedups of 50x or greater, so unfortunately numpy won’t be enough for modern deep learning.

- Here we introduce the most fundamental PyTorch concept: the Tensor. A PyTorch Tensor is conceptually identical to a numpy array: a Tensor is an n-dimensional array, and PyTorch provides many functions for operating on these Tensors. Behind the scenes, Tensors can keep track of a computational graph and gradients, but they’re also useful as a generic tool for scientific computing.

- Also unlike numpy, PyTorch Tensors can utilize GPUs to accelerate their numeric computations. To run a PyTorch Tensor on GPU, you simply need to specify the correct device.

- Here we use PyTorch Tensors to fit a third-order polynomial to a sine function. Like the numpy example above we need to manually implement the forward and backward passes through the network.

In [2]:
import torch
import math


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 2100.8466796875
199 1394.6785888671875
299 926.995849609375
399 617.220947265625
499 412.0144348144531
599 276.0612487792969
699 185.97816467285156
799 126.28038787841797
899 86.71295166015625
999 60.48365783691406
1099 43.09335708618164
1199 31.56139373779297
1299 23.912696838378906
1399 18.838687896728516
1499 15.471973419189453
1599 13.237480163574219
1699 11.754127502441406
1799 10.769144058227539
1899 10.114948272705078
1999 9.680317878723145
Result: y = 0.009218379855155945 + 0.8294650912284851 x + -0.0015903243329375982 x^2 + -0.0894506573677063 x^3
